In [3]:
# 함수형 API에서는 직접텐서들의 입출력을 다룸.
# 함수처럼 층을 사용하여 텐서를 입력받고 출력함

from keras import Input, layers

input_tensor = Input(shape=(32, ))
dense = layers.Dense(32, activation = 'relu')
output_tensor = dense(input_tensor)

In [2]:
# 간단한 예제를 통해 Sequential 모델과 함수형 API로 만든 모델 비교
from keras.models import Sequential, Model
from keras import Input
from keras import layers

# Sequential
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation = 'relu', input_shape = (64, )))
seq_model.add(layers.Dense(32, activation = 'relu'))
seq_model.add(layers.Dense(10, activation = 'softmax'))

# 함수형 API
input_tensor = Input(shape = (64, ))
x = layers.Dense(32, activation = 'relu')(input_tensor)
x = layers.Dense(32, activation = 'relu')(x)
output_tensor = layers.Dense(10, activation = 'softmax')(x)
model = Model(input_tensor, output_tensor)

Using TensorFlow backend.
W0726 22:28:20.204729 17332 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 22:28:20.295485 17332 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 22:28:20.312442 17332 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [4]:
# 모델이 같음을 확인할 수 있음.
model.summary()
seq_model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 

In [24]:
# 함수형API에서는 input_tensor 와 output_tensor의 모델 변환이 맞아야 함.
unRelated_tensor = Input(shape = (32, ))
Model(unRelated_tensor, output_tensor) # Graph disconnected

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_8:0", shape=(?, 64), dtype=float32) at layer "input_8". The following previous layers were accessed without issue: []

In [20]:
# Model 객체를 사용한 컴파일, 훈련, 평가 API는 Sequential 클래스 같습니다.
# optimizer     : rmsprop
# loss function : categorical_crossentropy
# 해당 compile 값을 기본으로, 위에서 만든 API 함수에 넣어보고, 모델 생성 후 평가해보기

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')

In [21]:
X_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

In [22]:
model.fit(X_train, y_train, epochs = 10, batch_size = 128)

Epoch 1/10
1000/1000 [==============================] - 1s 814us/step - loss: 11.5014
Epoch 2/10
1000/1000 [==============================] - 0s 52us/step - loss: 11.4966
Epoch 3/10
1000/1000 [==============================] - 0s 49us/step - loss: 11.4949
Epoch 4/10
1000/1000 [==============================] - 0s 48us/step - loss: 11.4939
Epoch 5/10
1000/1000 [==============================] - 0s 51us/step - loss: 11.4928
Epoch 6/10
1000/1000 [==============================] - 0s 46us/step - loss: 11.4915
Epoch 7/10
1000/1000 [==============================] - 0s 44us/step - loss: 11.4907
Epoch 8/10
1000/1000 [==============================] - 0s 46us/step - loss: 11.4898
Epoch 9/10
1000/1000 [==============================] - 0s 48us/step - loss: 11.4885
Epoch 10/10
1000/1000 [==============================] - 0s 44us/step - loss: 11.4877


In [23]:
score = model.evaluate(x_train, y_train)

1000/1000 [==============================] - 0s 293us/step


In [2]:
# 2개의 입력을 가진 질문-응답 모델의 함수형 API 구현하기.
# text_size = 1000
# question_size = 10000
# answer_vocabulary_size = 500

# 다중 입력 모델 만들기
from keras.models import Model
from keras import Input
from keras import layers

text_vocabulary_size = 1000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# input_text 생성
text_input = Input(shape = (None,), dtype = 'int32', name = 'text') # 이름은 text 라고 하는 Input_model 생성
embedded_text = layers.Embedding(
    text_vocabulary_size, 64)(text_input) # 입력을 크기가 64인 벡터의 크기로 임베딩
encoded_text  = layers.LSTM(32)(embedded_text)


question_input = Input(shape = (None,), dtype = 'int32', name = 'question') # 이름은 text 라고 하는 Input_model 생성
embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input) # 입력을 크기가 64인 벡터의 크기로 임베딩
encoded_question  = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question] ,axis = -1)  # 인코딩된 질문과 텍스트를 연결. axis -> -1 : 입력 값의 마지막 축

answer = layers.Dense(answer_vocabulary_size, activation = 'softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer = 'rmsprop',
             loss       = 'categorical_crossentropy',
             metrics    = ['acc']
             )



W0727 01:00:57.533487 14276 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 01:00:57.626231 14276 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 01:00:57.629223 14276 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 01:00:59.111261 14276 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 01:00:59.209995 14276 deprecation_wrapper.py:119] From C:\Users\koni1\Anac

In [12]:
# 다중 입력 모델에 데이터 주입하기
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length  = 100

# text, question input 에 필요한 데이터 생성
# 샘플 수 text: 1000개, length : 100, 1 ~ 1000.
#         question : 1000개, length : 1 ~ 10000.
text     = np.random.randint(1, text_vocabulary_size, size = (num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size = (num_samples, max_length)) 

# y_train data 만들기 (answer)
answers = np.random.randint(0, answer_vocabulary_size, size = num_samples)
answers = to_categorical(answers)
# model.fit
# epochs, batch_sizes
model.fit([text, question], answers, epochs = 10, batch_size = 128)                      # 1번째 방법. list 형식으로 input Data 전달
# model.fit({'text':text, 'question': question }, answers, epochs = 10, batch_size = 128)  # 2번째 방법. dict 형식으로 전달.

# model.evaluate 


W0727 01:05:57.492202 14276 deprecation.py:323] From C:\Users\koni1\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0727 01:06:01.554370 14276 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
1000/1000 [==============================] - 6s 6ms/step - loss: 6.2146 - acc: 0.0030
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.2022 - acc: 0.0300
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1869 - acc: 0.0270
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1202 - acc: 0.0070
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.0382 - acc: 0.0060
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.9836 - acc: 0.0060
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.9243 - acc: 0.0040
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.8697 - acc: 0.0130
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.8170 - acc: 0.0070
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.7655 - acc: 0.0120


In [17]:
# 3개의 출력을 가진 함수형 API 구현하기
# 소셜 미디어에서 익명 사용자의 포스트를 입력으로 받아 그 사람의 나이, 성별, 소득 수준 등을 예측하는 모델 생성

# Model format
# Embedding 층
# Conv1D -> 5층~
# MaxPooling1D -> 2층~ 
# GlobalMaxPooing1D -> 1층~ 
# Dense -> 1층~

from keras import layers
from keras import Input
from keras.models import Model
 
vocabulary_size = 50000 # input data 단어 size
num_income_groups = 10  # 소득 수준 1 ~ 10 

posts_input    = Input(shape = (None,), dtype = 'int32', name = 'posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation = 'relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 출력 층에 이름을 부여하고, 알맞는 출력 형태를 지정하자.
age_prediction    = layers.Dense(1, name = 'age')(x)    # 나이
income_prediction = layers.Dense(num_income_groups, activation = 'softmax', name = 'income')(x)
gender_prediction = layers.Dense(1, activation = 'sigmoid', name = 'gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

W0727 01:31:53.185845 14276 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [18]:
# 출력이 3개 이므로, 모델 compile 시, loss function을 3개 지정 해주야 함.
model.compile(
    optimizer = 'rmsprop',
    loss      = ['mse', 'categorical_crossentropy', 'binary_crossentropy']
)

model.compile(
    optimizer = 'rmsprop',
    loss = {
        'age'    : 'mse',
        'income' : 'categorical_crossentropy',
        'gender' : 'binary_crossentropy'
    }
)

In [20]:
# 다중 출력 시, 가중치를 지정할 수 있음.
model.compile(
    optimizer = 'rmsprop',
    loss = {
        'age'    : 'mse',
        'income' : 'categorical_crossentropy',
        'gender' : 'binary_crossentropy'
    },
    
    loss_weights = {
        'age' : 0.25,
        'income' : 1.,
        'gender' : 10.
    }
)

In [22]:
# 인셉션 V3(Inception V3) 모델 함수형 API로 만들어보기.
# x는 4D Data라고 가정.
from keras import layers
from keras import Input
from keras.models import Model

branch_1 = layers.Conv2D(128, 1, activation = 'relu', strides = 2)(x)

branch_2 = layers.Conv2D(128, 1, activation = 'relu')(x)
branch_2 = layers.Conv2D(128, 3, activation = 'relu', strides = 2)(branch_2)

branch_3 = layers.Average2D(128, 3, activation = 'relu', strides = 2)(x)
branch_3 = layers.Conv2D(128 , 3, activation = 'relu')(branch_3)

branch_4 = layers.Conv2D(128, 1, activation = 'relu')(x)
branch_4 = layers.Conv2D(128 , 3, activation = 'relu')(branch_4)
branch_4 = layers.Average2D(128, 3, activation = 'relu', strides = 2)(branch_4)

output = layers.concatenate([branch_1, branch_2, branch_3, branch_4], axis = -1)

ValueError: Input 0 is incompatible with layer conv2d_2: expected ndim=4, found ndim=2

In [ ]:
 # 잔차 연결(Residual Connection)

x = ...
y = layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(x)
..
y = layers.add([y, x])

residual = layers.Conv2D(128, 1, strides = 2, padding = 'same')(x)
y = layers.add([y, residual])

In [1]:
# 층 가중치 공유 예제
# 두 문장이 유사한지, 유사하지 않는지를 확인하는 예제.
# 왜 가중치 공유를 해야하는가?
# -> A와 B의 유사도는 B와 A의 유사도와 같음. 즉 두 개의 모델을 각각 만들 필요가 없고, A,B가 들어왔을 때 같은지, 같지 않은지만
#    확인하면 됨.

from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input  = Input(shape = (None, 128))  # 크기가 128인 벡터의 가변 길이 시퀀스
left_output = lstm(left_input)

right_input  = Input(shape = (None, 128)) # 크기가 128인 벡터의 가변 길이 시퀀스
right_output = lstm(right_input)

# concatenate function으로 노드 병합
merged = layers.concatenate([left_input, right_input], axis = -1) # 왜 axis = -1 인가?
predictions = layers.Dense(1, activation = 'sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)



Using TensorFlow backend.
W0727 17:12:16.614800 18252 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 17:12:16.653696 18252 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 17:12:16.664670 18252 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



NameError: name 'left_data' is not defined

In [2]:
# 층과 모델.
# 층 내에 모델 자체가 들어갈 수 있음.
# y = model(x) .            #
# y1, y2 = model([x1, x2])  # 

from keras import layers
from keras import Input
from keras import applications # Xception 모델을 호출하기 위한 import

xception_base = applications.Xception(weights = None, include_top = False)

left_input  = Input(shape = (250, 250, 3)) # 250, 250, 3 RGB
right_input = Input(shape = (250, 250, 3))

# 같은 비전 모델이 두번 사용.
left_features  = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis = -1)

In [4]:
# 콜백 함수 예시
# 1. EarlyStopping 과 ModelCheckPoint param 설정
import keras

callbacks_list = [
    
    keras.callbacks.EarlyStopping( # 성능 향상이 멈추면 훈련을 중지하게끔 하는 함수
        monitor  = 'val_acc', # 모델의 검증 정확도 모니터링
        patience = 1,
    ),
    
    keras.callbacks.ModelCheckpoint(
        filepath = 'my_model.h5', # 모델 파일의 경로
        monitor  = 'val_loss',
        save_best_only = True, # monitor, save_best_only parameter 값을 통해 
                               # 해당 val_loss 값이 좋아지지 않으면 모델을 저장하지 않겠다는 의미
    )
]

# 2. model.compile 생성
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['acc'])

model.fit(x, y,
          epochs = 10,
          batch_size = 32,
          callbacks  = callbacks_list,       # --> model.fit 함수에서 callbacks param 값에 callback 함수 전달
          validation_data = (x_val, y_val))  # --> validation_data에 값이 없으면 안됨

W0727 17:48:40.619823 18252 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 17:48:40.660715 18252 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0727 17:48:40.671685 18252 deprecation.py:323] From C:\Users\koni1\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


NameError: name 'x' is not defined

In [5]:
# ReduceLROnPlateau
# -> 학습 손실이 개선되지 않을 때, 자동으로 학습률을 조정.
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor  = 'val_loss',
        factor   = 0.1,       # --> 콜백이 호출될 때 학습률을 10배로 줄임. : 학습률 x 0.1 
        patience = 10,        # --> 검증 손실이 10 에포크 동안 좋아지지 않으면 콜백이 호출.
    )
]

model.fit(x, y,
          epoch      = 10,
          batch_size = 32,
          callbacks  = callbacks_list, 
          validation_data = (x_val, y_val))

NameError: name 'x' is not defined

In [ ]:
# 자신만의 콜백을 만들기.
# 매 에포크의 끝에서 검증 세트의 첫 번째 샘플로 모델에 있는 
# 모든 층의 활성화 출력을 계산하여 디스크에 저장하는 자작 콜백의 예.

class ActivationLogger(keras.callbacks.Callback):
    
    def set_model(self, model): #  호출하는 모델에 대한 정보를 전달하기 위해 훈련하기 전에 호출 됨.
        self.model = model
        layer_outputs = [layer.output for layer in model.layers ]               # 해당 층의 활성화 출력을 array로 만듬.
        self.activations_model = keras.models.Model(model.input, layer_outputs) # 각 층의 활성화 출력을 반환하는 Model 객체. 
        
    
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
            
        validation_sample = self.validation_data[0][0:1]                       # 검증데이터의 첫 번째 샘플을 가져옵니다.
        actiavtions       = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_') + str(epoch) + '.npz', 'wb')         # 배열을 디스크에 저장. 
        np.savez(f, activations)           
        f.close()
        
        

In [6]:
# IMDB 감성 분석 문제 
# !D 컨브넷 훈련
# IMDB 어휘 사전에서 빈도가 높은 2000개의 단어 사용


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [1]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000 # 특성으로 사용할 단어의 수
max_len      = 500  # 사용할 텍스트의 길이(가장 빈번한 max_feature개의 단어만 사용)

# train : 25000
# test  : 25000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features) # 빈도가 높은 2000개의 단어만 사용하여 만든 input data.
x_train = sequence.pad_sequences(x_train, maxlen = max_len) 
x_test  = sequence.pad_sequences(x_test, maxlen = max_len) 

Using TensorFlow backend.


In [2]:
model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length = max_len, name = 'embed'))
model.add(layers.Conv1D(32, 7, activation = 'relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation = 'relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['acc'])

W0727 20:28:18.966359  5724 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 20:28:19.044150  5724 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 20:28:19.054124  5724 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 20:28:19.220679  5724 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0727 20:28:19.372272  5724 deprecation_wrapper.py:119] From C:\Users\koni1\Ana

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 텐서보드 콜백과 함께 모델 훈련
callbacks = [
    keras.callbacks.TensorBoard(
    log_dir         = 'my_log_dir',       # 로그 파일이 기록될 위치
    histogram_freq  = 1,                  # 1 에포크마다 활성화 출력의 히스토그램을 기록
    embeddings_freq = 1,                  # 1 에포크마다 임베딩 데이터 기록.
    embeddings_data = x_train[:100]       # 책에는 없지만 추가 해야함.! 
    )
]

In [9]:
history = model.fit(x_train, y_train,
                    epochs = 20,
                    batch_size = 128,
                    validation_split = 0.2,
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples


W0727 20:41:24.889057  5724 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\callbacks.py:887: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



Epoch 1/20
20000/20000 [==============================] - 53s 3ms/step - loss: 0.2943 - acc: 0.7129 - val_loss: 0.6450 - val_acc: 0.6440
Epoch 2/20
20000/20000 [==============================] - 49s 2ms/step - loss: 0.2491 - acc: 0.6616 - val_loss: 0.6821 - val_acc: 0.5874
Epoch 3/20
20000/20000 [==============================] - 52s 3ms/step - loss: 0.2298 - acc: 0.5833 - val_loss: 0.7362 - val_acc: 0.5084
Epoch 4/20
20000/20000 [==============================] - 49s 2ms/step - loss: 0.1799 - acc: 0.5508 - val_loss: 0.9541 - val_acc: 0.4566
Epoch 5/20
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1637 - acc: 0.4848 - val_loss: 0.9808 - val_acc: 0.4212
Epoch 6/20
20000/20000 [==============================] - 48s 2ms/step - loss: 0.1402 - acc: 0.4263 - val_loss: 1.0727 - val_acc: 0.3696


W0727 20:49:19.584925  5724 deprecation.py:323] From C:\Users\koni1\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


Epoch 7/20
20000/20000 [==============================] - 48s 2ms/step - loss: 0.1225 - acc: 0.3657 - val_loss: 0.9941 - val_acc: 0.3494
Epoch 8/20
20000/20000 [==============================] - 50s 3ms/step - loss: 0.1141 - acc: 0.3145 - val_loss: 1.2332 - val_acc: 0.2938
Epoch 9/20
20000/20000 [==============================] - 47s 2ms/step - loss: 0.1125 - acc: 0.2644 - val_loss: 1.1109 - val_acc: 0.2952
Epoch 10/20
20000/20000 [==============================] - 46s 2ms/step - loss: 0.1101 - acc: 0.2386 - val_loss: 1.1412 - val_acc: 0.2828
Epoch 11/20
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1219 - acc: 0.2013 - val_loss: 1.1940 - val_acc: 0.2602
Epoch 12/20
20000/20000 [==============================] - 52s 3ms/step - loss: 0.1056 - acc: 0.1813 - val_loss: 1.2563 - val_acc: 0.2430
Epoch 13/20
20000/20000 [==============================] - 47s 2ms/step - loss: 0.1063 - acc: 0.1663 - val_loss: 1.2766 - val_acc: 0.2394
Epoch 14/20
20000/20000 [============

In [11]:
# keras.utils.plot_model 유틸리티 사용하여 그래프 그려보기
from keras.utils import plot_model
plot_model(model, show_shapes = True, to_file = 'model.png') # show_shape : 모델 층의 크기 정보 추가.


ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [ ]:
# 작은 데이터셋에서 이미지 분류 문제를 위한 가벼운 깊이 분리 컨브넷을 만드는 예제.
